In [ ]:
from __future__ import annotations

import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize

from example_models import get_linear_chain_2v
from mxlpy import Simulator, fit_local, make_protocol, plot, unwrap

## Fitting

Almost every model at some point needs to be fitted to experimental data to be **validated**.  

*mxlpy* offers highly customisable local and global routines for fitting either **time series** or **steady-states**.  

<img src="assets/fitting.png" style="max-height: 175px;" />

For this tutorial we are going to use the `fit` module to optimise our parameter values and the `plot` module to plot some results.  

Let's get started!

## Creating synthetic data

Normally, you would fit your model to experimental data.  
Here, for the sake of simplicity, we will generate some synthetic data.  

Checkout the [basics tutorial](basics.ipynb) if you need a refresher on building and simulating models.  

In [ ]:
# As a small trick, let's define a variable for the model function
# That way, we can re-use it all over the file and easily replace
# it with another model
model_fn = get_linear_chain_2v

res = unwrap(
    Simulator(model_fn())
    .update_parameters({"k1": 1.0, "k2": 2.0, "k3": 1.0})
    .simulate_time_course(np.linspace(0, 10, 101))
    .get_result()
).get_combined()

fig, ax = plot.lines(res)
ax.set(xlabel="time / a.u.", ylabel="Conc. & Flux / a.u.")
plt.show()

### Steady-states

For the steady-state fit we need two inputs:

1. the steady state data, which we supply as a `pandas.Series`
2. an initial parameter guess

The fitting routine will compare all data contained in that series to the model output.  

> Note that the data both contains concentrations and fluxes!  

In [ ]:
data = res.iloc[-1]
data.head()

In [ ]:
fit_result = unwrap(
    fit_local.steady_state(
        model_fn(),
        p0={"k1": 1.038, "k2": 1.87, "k3": 1.093},
        data=res.iloc[-1],
    )
)

fit_result.best_pars

If only some of the data is required, you can use a subset of it.  
The fitting routine will only try to fit concentrations and fluxes contained in that series.

In [ ]:
fit_result = unwrap(
    fit_local.steady_state(
        model_fn(),
        p0={"k1": 1.038, "k2": 1.87, "k3": 1.093},
        data=data.loc[["x", "y"]],
    )
)
fit_result.best_pars

### Time course

For the time course fit we need again need two inputs

1. the time course data, which we supply as a `pandas.DataFrame`
2. an initial parameter guess

The fitting routine will create data at every time points specified in the `DataFrame` and compare all of them.  

Other than that, the same rules of the steady-state fitting apply.  

In [ ]:
fit_result = unwrap(
    fit_local.time_course(
        model_fn(),
        p0={"k1": 1.038, "k2": 1.87, "k3": 1.093},
        data=res,
        minimizer=fit_local.ScipyMinimizer(
            tol=1e-6,
            method="Nelder-Mead",
        ),
    )
)

fit_result.best_pars

## Protcol time courses


Normally, you would fit your model to experimental data.  
Here, again, for the sake of simplicity, we will generate some synthetic data.  

In [ ]:
protocol = make_protocol(
    [
        (1, {"k1": 1.0}),
        (1, {"k1": 2.0}),
        (1, {"k1": 1.0}),
    ]
)

res_protocol = unwrap(
    Simulator(model_fn())
    .update_parameters({"k1": 1.0, "k2": 2.0, "k3": 1.0})
    .simulate_protocol(
        protocol,
        time_points_per_step=10,
    )
    .get_result()
).get_combined()

fig, ax = plot.lines(res_protocol)
ax.set(xlabel="time / a.u.", ylabel="Conc. & Flux / a.u.")
plt.show()

For the protocol time course fit we need three inputs

1. an initial parameter guess
2. the time course data, which we supply as a `pandas.DataFrame`
3. the protocol, which we supply as a `pandas.DataFrame`

> Note that the parameter given by the protocol cannot be fitted anymore  


In [ ]:
fit_result = unwrap(
    fit_local.protocol_time_course(
        model_fn(),
        p0={"k2": 1.87, "k3": 1.093},  # note that k1 is given by the protocol
        data=res_protocol,
        protocol=protocol,
    )
)

fit_result.best_pars

<div style="color: #ffffff; background-color: #04AA6D; padding: 3rem 1rem 3rem 1rem; box-sizing: border-box">
    <h2>First finish line</h2>
    With that you now know most of what you will need from a day-to-day basis about fitting in mxlpy.
    <br />
    <br />
    Congratulations!
</div>

## Advanced topics / customisation


All fitting routines internally are build in a way that they will call a tree of functions. 

- `minimizer`
  - `residual_fn`
    - `integrator`
    - `loss_fn`
  

You can therefore use dependency injection to overwrite the minimisation function, the loss function, the residual function and the integrator if need be.  

In [ ]:
from functools import partial
from typing import TYPE_CHECKING, cast

from mxlpy.fit.common import LossFn
from mxlpy.integrators import Scipy

if TYPE_CHECKING:
    import pandas as pd

    from mxlpy.fit.common import ResidualFn
    from mxlpy.model import Model
    from mxlpy.types import Array, IntegratorType

## Parameterising scipy optimise

In [ ]:
optimizer = fit_local.ScipyMinimizer(tol=1e-6, method="Nelder-Mead")

### Custom loss function

You can change the loss function that is being passed to the minimsation function using the `loss_fn` keyword.  
Depending on the use case (time course vs steady state) this function will be passed two pandas `DataFrame`s or `Series`.

In [ ]:
def mean_absolute_error(
    x: pd.DataFrame | pd.Series,
    y: pd.DataFrame | pd.Series,
) -> float:
    """Mean absolute error between two dataframes."""
    return cast(float, np.mean(np.abs(x - y)))


fit_result = unwrap(
    fit_local.time_course(
        model_fn(),
        p0={"k1": 1.038, "k2": 1.87, "k3": 1.093},
        data=res,
        loss_fn=mean_absolute_error,
    )
)

fit_result.best_pars

### Custom integrator

You can change the default integrator to an integrator of your choice by partially application of the class of any of the existing ones.  

Here, for example, we choose the `Scipy` solver suite and set the default relative and absolute tolerances to `1e-6` respectively.

In [ ]:
fit_result = unwrap(
    fit_local.time_course(
        model_fn(),
        p0={"k1": 1.038, "k2": 1.87, "k3": 1.093},
        data=res,
        integrator=partial(Scipy, rtol=1e-6, atol=1e-6),
    )
)

fit_result.best_pars

### Custom minimisation

You can change the default `minimizer` from to any other function that takes a `ResidualFn` and minimizes it.   

In [ ]:
from mxlpy.fit.local_ import Bounds, MinResult


def nelder_mead(
    residual_fn: ResidualFn,
    p0: dict[str, float],
    bounds: Bounds,
) -> MinResult | None:
    res = minimize(
        residual_fn,
        x0=list(p0.values()),
        bounds=[bounds.get(name, (1e-6, 1e6)) for name in p0],
        method="Nelder-Mead",
    )
    if res.success:
        return MinResult(
            parameters=dict(
                zip(
                    p0,
                    res.x,
                    strict=True,
                )
            ),
            residual=res.fun,
        )
    return None


fit_result = unwrap(
    fit_local.time_course(
        model_fn(),
        p0={"k1": 1.038, "k2": 1.87, "k3": 1.093},
        data=res,
        minimizer=nelder_mead,
    )
)

fit_result.best_pars

### Custom residual function

You can change the residual function to include further behaviour.  

The barebones implementation is given below

In [ ]:
def time_course_residual(
    par_values: Array,
    # This will be filled out by partial
    par_names: list[str],
    data: pd.DataFrame,
    model: Model,
    y0: dict[str, float] | None,
    integrator: IntegratorType,
    loss_fn: LossFn,
) -> float:
    """Calculate residual error between model time course and experimental data.

    Args:
        par_values: Parameter values to test
        data: Experimental time course data
        model: Model instance to simulate
        y0: Initial conditions
        par_names: Names of parameters being fit
        integrator: ODE integrator class to use
        loss_fn: Loss function to use for residual calculation

    Returns:
        float: Root mean square error between model and data

    """
    res = (
        Simulator(
            model.update_parameters(dict(zip(par_names, par_values, strict=True))),
            y0=y0,
            integrator=integrator,
        )
        .simulate_time_course(cast(list, data.index))
        .get_result()
    )
    if res is None:
        return cast(float, np.inf)
    results_ss = res.get_combined()

    return loss_fn(
        results_ss.loc[:, cast(list, data.columns)],
        data,
    )


fit_result = unwrap(
    fit_local.time_course(
        model_fn(),
        p0={"k1": 1.038, "k2": 1.87, "k3": 1.093},
        data=res,
        residual_fn=time_course_residual,
    )
)

fit_result.best_pars